In [2]:
## Import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, matthews_corrcoef
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import seaborn as sns
import pandas as pd
import csv
import math
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score,KFold,cross_val_predict
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
import sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NP/MRMD_BTG/Nps-train_features.csv')

In [ ]:
from sklearn.preprocessing import StandardScaler
# Initialise the Scaler
scaler = StandardScaler()

# To scale data
scaler.fit(df)

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(999, inplace=True)

In [ ]:
# Split the dataset into features (X) and labels (y)
X = df.iloc[:, 0:170].values
y = df.iloc [:, 170].values
print(X.shape)
print(y.shape)

In [ ]:
# Split into training and test set
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.2, random_state=42)
print (len(X_train),len(X_test),len(y_train),len(y_test))

In [ ]:
import numpy as np
import tensorflow as tf
from keras import layers, models, optimizers, losses, callbacks

# Define a custom Capsule layer
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.num_capsules * self.dim_capsules),
                                      initializer='glorot_uniform',
                                      name='kernel')

    def call(self, inputs):
        inputs_hat = tf.linalg.matmul(inputs, self.kernel)
        inputs_hat = tf.reshape(inputs_hat, (-1, inputs.shape[1], self.num_capsules, self.dim_capsules))
        inputs_hat = tf.transpose(inputs_hat, perm=[0, 2, 1, 3])

        b = tf.zeros_like(inputs_hat[:, :, :, 0])
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=1)
            s = tf.reduce_sum(c[..., tf.newaxis] * inputs_hat, axis=2)
            v = self.squash(s)
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * v[:, :, tf.newaxis, :], axis=-1)
        return v

    def squash(self, s):
        s_squared_norm = tf.reduce_sum(tf.square(s), axis=-1, keepdims=True)
        scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + tf.keras.backend.epsilon())
        return scale * s

# Define the Capsule Network model
input_shape = (X_train.shape[1], 1)
inputs = layers.Input(shape=input_shape)

# Initial Conv1D layer
x = layers.Conv1D(64, 1, activation='relu')(inputs)
x = layers.MaxPooling1D(pool_size=2)(x)

# Second Conv1D layer
x = layers.Conv1D(64, 6, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)
# Third Conv1D layer
x = layers.Conv1D(32, 1, activation='relu')(x)
x = layers.MaxPooling1D(pool_size=2)(x)

# Flatten layer
x = layers.Flatten()(x)

# Primary Capsule layer
x = layers.Dense(128)(x)
x = layers.Reshape((-1, 8))(x)

# Capsule layer
capsules = CapsuleLayer(num_capsules=10, dim_capsules=20)(x)

# Output layer
capsules = layers.Flatten()(capsules)
outputs = layers.Dense(1, activation='sigmoid')(capsules)

model_capsnet = models.Model(inputs, outputs)

# Compile the model
model_capsnet.compile(optimizer=optimizers.RMSprop(),
                      loss=losses.BinaryCrossentropy(),
                      metrics=['accuracy'])

model_capsnet.summary()

# Initialize lists to store evaluation metrics for each fold
accuracy_list = []
f1_list = []
precision_list = []
recall_list = []
sensitivity_list = []
specificity_list = []
mcc_list = []
auc_list = []  # Initialize a list for AUC values

# Perform 5-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=5)
for train_index, val_index in kfold.split(X_train):
    # Split the data into training and validation sets for the current fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train the model on the current fold
    history=model_capsnet.fit(X_train_fold, y_train_fold, epochs=40, batch_size=64, verbose=0,validation_split=0.1)

    # Evaluate the model on the validation set for the current fold
    y_val_pred = model_capsnet.predict(X_val_fold)
    y_val_pred_binary = (y_val_pred > 0.5).astype(int)

    # Calculate confusion matrix for the current fold
    cm = confusion_matrix(y_val_fold, y_val_pred_binary)
    # Calculate AUC for the current fold
    auc = roc_auc_score(y_val_fold, y_val_pred)  # Use predicted probabilities for AUC calculation
    auc_list.append(auc)


    # Calculate performance evaluation metrics for the current fold
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]

    # Accuracy
    accuracy = (TP + TN) / float(TP + TN + FP + FN)
    accuracy_list.append(accuracy)

    # F1-Score
    f1 = 2 * TP / float(2 * TP + FP + FN)
    f1_list.append(f1)

    # Precision
    precision = TP / float(TP + FP)
    precision_list.append(precision)

    # Recall
    recall = TP / float(TP + FN)
    recall_list.append(recall)

    # Sensitivity
    sensitivity = TP / float(TP + FN)
    sensitivity_list.append(sensitivity)

    # Specificity
    specificity = TN / float(TN + FP)
    specificity_list.append(specificity)

    # MCC (Matthews Correlation Coefficient)
    mcc = ((TP * TN) - (FP * FN)) / float((np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))) or 1)
    mcc_list.append(mcc)

# Calculate average performance evaluation metrics across all folds
avg_accuracy = np.mean(accuracy_list)
avg_f1 = np.mean(f1_list)
avg_precision = np.mean(precision_list)
avg_recall = np.mean(recall_list)
avg_sensitivity = np.mean(sensitivity_list)
avg_specificity = np.mean(specificity_list)
avg_mcc = np.mean(mcc_list)

# Calculate average AUC across all folds
avg_auc = np.mean(auc_list)
# Print average performance evaluation metrics
print("Accuracy =", avg_accuracy)
print("F1 Score =", avg_f1)
print("Precision =", avg_precision)
print("Recall =", avg_recall)
print("Sensitivity =", avg_sensitivity)
print("Specificity =", avg_specificity)
print("MCC =", avg_mcc)
print("AUC =", avg_auc)
